In [2]:
import pandas as pd

In [29]:
dfs = []
for i in range(142):
    path = f"C:/Users/gaolg/OneDrive/Área de Trabalho/IPS Nordeste/bases_tratadas/alimentacao/alimentacao_2021.parquet/part.{i}.parquet"
    df = pd.read_parquet(path)
    df["DS_IMC"] = df["DS_IMC"].dropna().apply(lambda x: x.replace(",",".")).astype(float)
    df= df[["CO_MUNICIPIO_IBGE", "DS_IMC"]]
    df = df[["CO_MUNICIPIO_IBGE", "DS_IMC"]].query("DS_IMC<25").groupby("CO_MUNICIPIO_IBGE").count().reset_index()
    df = df.rename(columns={"DS_IMC":"sub_nutricao", "CO_MUNICIPIO_IBGE":"cod_municipio"})
    dfs.append(df)
    del df

alimentacao = pd.concat(dfs)
alimentacao["cod_municipio"] = alimentacao["cod_municipio"].astype(str)
alimentacao = alimentacao.groupby("cod_municipio").sum().reset_index()

In [57]:
subnutricao = pd.merge(alimentacao, populacao, on="cod_municipio", how="left")
subnutricao["sub_nutricao_porcentagem"] = subnutricao["sub_nutricao"]/subnutricao["pop_2021"]

subnutricao.to_csv("C:/Users/gaolg/OneDrive/Área de Trabalho/IPS Nordeste/bases_tratadas/subnutricao.csv", index=False)

#### Abastecimento água

In [27]:
path_to_output = "C:/Users/gaolg/OneDrive/Área de Trabalho/projetos/ips_nordeste/Indicadores/primeiros"

In [5]:
import pandas as pd

agua_h_raw = pd.read_excel("C:/Users/gaolg/OneDrive/Área de Trabalho/projetos/ips_nordeste/Necessidades Humanas Básicas/agua/agua_serie_historica.xlsx")

In [6]:
agua_h = agua_h_raw[["Código do Município", "Ano de Referência", "AG001 - População total atendida com abastecimento de água"]]
agua_h = agua_h.rename(columns={"Código do Município":"cod_municipio", "Ano de Referência":"ano", "AG001 - População total atendida com abastecimento de água":"pop_agua"})

In [8]:
agua_h = agua_h.query("ano==2021").groupby("cod_municipio", as_index=False).sum()

In [9]:
agua_h = agua_h[["cod_municipio", "pop_agua"]]
agua_h["cod_municipio"] = agua_h["cod_municipio"].astype(str)

In [11]:
paths_inputs = "C:/Users/gaolg/OneDrive/Área de Trabalho/projetos/ips_nordeste/bases_tratadas/"
populacao = pd.read_parquet(paths_inputs+"populacao_tratados.parquet")
populacao = populacao[["cod_municipio", "pop_2021"]]

In [17]:
agua_h.rename(columns={"cod_municipio":"cod_municipio_6"}, inplace=True)

In [16]:
populacao["cod_municipio_6"] = populacao["cod_municipio"].apply(lambda x: x[:6])

In [21]:
agua_h = agua_h.merge(populacao, on="cod_municipio_6", how="left").drop(columns=["cod_municipio_6"])

In [28]:
agua_h["pop_agua_porcentagem"] = agua_h["pop_agua"]/agua_h["pop_2021"]
agua_h.to_csv(path_to_output +"abastecimento_agua.csv", index=False)

## Indicadores cad_unico

In [114]:
path_to_cad = "C:/Users/gaolg/OneDrive/Área de Trabalho/IPS Nordeste/Necessidades Humanas Básicas/cad_unico/"
cad = pd.read_csv(path_to_cad+"cad_unico_total.csv",  encoding="latin1")
cad = cad[["Código", "Referência", "Quantidade total de famílias cadastradas" ]]
cad = cad.rename(columns={"Código":"cod_municipio",
                     "Referência":"ano",
                        "Quantidade total de famílias cadastradas":"familias_cadastradas"})

cad["ano"] =  cad["ano"].apply(lambda x: int(x[-4:]))

cad = cad.query("ano==2023")
cad = cad[["cod_municipio", "familias_cadastradas"]]
cad = cad.groupby("cod_municipio").sum().reset_index()

C:\Users\gaolg\AppData\Local\Temp\ipykernel_9924\704081134.py:2: DtypeWarning: Columns (18,19) have mixed types. Specify dtype option on import or set low_memory=False.
  cad = pd.read_csv(path_to_cad+"cad_unico_total.csv",  encoding="latin1")


##### Agua

In [150]:
path_to_output = "C:/Users/gaolg/OneDrive/Área de Trabalho/IPS Nordeste/Indicadores/primeiros/"

In [141]:
agua = pd.read_csv(path_to_cad+"/agua.csv", encoding="latin1")
agua.rename(columns={"Código":"cod_municipio",
                     "Referência":"ano",
                     agua.columns[4]:"rede_geral",
                     agua.columns[5]:"poco_nascente",
                     agua.columns[6]:"cisterna"}, inplace=True)

agua = agua[["cod_municipio", "ano", "rede_geral", "poco_nascente", "cisterna"]]
agua["ano"] =  agua["ano"].apply(lambda x: int(x[-4:]))

agua = agua.query("ano==2023")
agua["agua_total"] = agua["rede_geral"]+agua["poco_nascente"]+agua["cisterna"]

agua = agua[["cod_municipio", "agua_total"]]
agua = agua.groupby("cod_municipio").sum().reset_index()

In [142]:
agua = agua.merge(cad, on="cod_municipio", how="left")
agua["agua_porcentagem"] = agua["agua_total"]/agua["familias_cadastradas"]

In [143]:
agua.to_csv(path_to_output + "cad_unico_agua.csv", index=False)

#### iluminicacao

In [126]:
iluminacao = pd.read_csv(path_to_cad+"/iluminacao.csv", encoding="latin1")
iluminacao.rename(columns={"Código":"cod_municipio",
                        "Referência":"ano",
                        iluminacao.columns[4]:"medidor_proprio",
                        iluminacao.columns[5]:"medidior_geral",
                        iluminacao.columns[6]:"sem_medidor"},
                inplace=True)

iluminacao = iluminacao[["cod_municipio", "ano", "medidor_proprio", "medidior_geral", "sem_medidor"]]
iluminacao["ano"] =  iluminacao["ano"].apply(lambda x: int(x[-4:]))
iluminacao = iluminacao.query("ano==2023")

iluminacao["iluminacao_total"] = iluminacao["medidor_proprio"]+iluminacao["medidior_geral"]+iluminacao["sem_medidor"]
iluminacao = iluminacao[["cod_municipio", "iluminacao_total"]]
iluminacao = iluminacao.groupby("cod_municipio").sum().reset_index()

In [128]:
iluminacao = iluminacao.merge(cad, on="cod_municipio", how="left")
iluminacao["iluminacao_porcentagem"] = iluminacao["iluminacao_total"]/iluminacao["familias_cadastradas"]

In [130]:
iluminacao.to_csv("cad_unico_iluminacao.csv", index=False)

#### sanitario

In [152]:
sanitario = pd.read_csv(path_to_cad+"/sanitario.csv", encoding="latin1")

FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/gaolg/OneDrive/Área de Trabalho/IPS Nordeste/Indicadores/primeiros/cad_unico_sanitario.csv'

In [147]:
sanitario[[sanitario.columns[4], sanitario.columns[5], sanitario.columns[11], sanitario.columns[12]]]

,Quantidade de famílias NÃO beneficiárias do Programa Bolsa Família cadastradas em domicílios com escoamento sanitário por rede coletora de esgoto ou pluvial,Quantidade de famílias NÃO beneficiárias do Programa Bolsa Família cadastradas em domicílios com escoamento sanitário por fossa séptica,Quantidade de famílias BENEFICIÁRIAS do Programa Bolsa Família cadastradas em domicílios com escoamento sanitário por rede coletora de esgoto ou pluvial,Quantidade de famílias BENEFICIÁRIAS do Programa Bolsa Família cadastradas em domicílios com escoamento sanitário por fossa séptica
0,13,606,10,989
1,12,701,10,961
2,46,950,23,1045
3,14,668,14,987
4,309,844,162,1049
...,...,...,...,...
389895,15,6,15,7
389896,15,6,17,8
389897,16,10,15,11
389898,17,9,14,11


In [148]:
sanitario = pd.read_csv(path_to_cad+"/sanitario.csv", encoding="latin1")
sanitario.rename(columns={"Código":"cod_municipio",
                        "Referência":"ano",
                        sanitario.columns[4]:"esgoto_1",
                        sanitario.columns[5]:"esgoto_2",
                        sanitario.columns[11]:"esgoto_3",
                        sanitario.columns[12]:"esgoto_4"},
                        inplace=True)
sanitario = sanitario[["cod_municipio", "ano", "esgoto_1", "esgoto_2", "esgoto_3", "esgoto_4"]]

sanitario["ano"] =  sanitario["ano"].apply(lambda x: int(x[-4:]))
sanitario = sanitario.query("ano==2023")

sanitario["sanitario_total"] = sanitario["esgoto_1"]+sanitario["esgoto_2"]+sanitario["esgoto_3"]+sanitario["esgoto_4"]
sanitario = sanitario[["cod_municipio", "sanitario_total"]]
sanitario = sanitario.groupby("cod_municipio").sum().reset_index()

In [149]:
sanitario = sanitario.merge(cad, on="cod_municipio", how="left")
sanitario["sanitario_porcentagem"] = sanitario["sanitario_total"]/sanitario["familias_cadastradas"]

,cod_municipio,sanitario_total,familias_cadastradas,sanitario_porcentagem
0,110001,567,14809.0,0.038288
1,110002,3209,51741.0,0.062020
2,110003,485,2731.0,0.177591
3,110004,36687,52768.0,0.695251
4,110005,1895,9489.0,0.199705
...,...,...,...,...
5565,522200,473,7927.0,0.059669
5566,522205,2415,5995.0,0.402836
5567,522220,902,4420.0,0.204072
5568,522230,144,5004.0,0.028777


In [153]:
sanitario.to_csv(path_to_output + "cad_unico_sanitario.csv", index=False)

#### Lixo

In [154]:
lixo = pd.read_csv(path_to_cad+"/lixo.csv", encoding="latin1")
lixo.columns

Index(['Código', 'Unidade Territorial', 'UF', 'Referência',
       'Quantidade de famílias cadastradas em domicílios onde o lixo é coletado diretamente',
       'Quantidade de famílias cadastradas em domicílios onde o lixo é coletado indiretamente',
       'Quantidade de famílias cadastradas em domicílios onde o lixo é queimado ou enterrado na propriedade',
       'Quantidade de famílias cadastradas em domicílios onde o lixo é jogado em terreno baldio ou logradouro (rua, avenida, etc)',
       'Quantidade de famílias cadastradas em domicílios onde o lixo é jogado em rio ou mar',
       'Quantidade de famílias cadastradas em domicílios com outras formas de destinação do lixo',
       'Quantidade de famílias cadastradas em domicílios sem informação sobre a forma de destinação do lixo'],
      dtype='object')

In [161]:
lixo = pd.read_csv(path_to_cad+"/lixo.csv", encoding="latin1")
lixo.rename(columns={"Código":"cod_municipio",
                        "Referência":"ano",
                        lixo.columns[4]:"coleta_1",
                        lixo.columns[5]:"coleta_2"},
                        inplace=True)
lixo = lixo[["cod_municipio", "ano", "coleta_1", "coleta_2"]]
lixo["ano"] =  lixo["ano"].apply(lambda x: int(x[-4:]))
lixo = lixo.query("ano==2023")
lixo["lixo_total"] = lixo["coleta_1"]+lixo["coleta_2"]
lixo = lixo[["cod_municipio", "lixo_total"]]
lixo = lixo.groupby("cod_municipio").sum().reset_index()


In [162]:
lixo = lixo.merge(cad, on="cod_municipio", how="left")
lixo["lixo_porcentagem"] = lixo["lixo_total"]/lixo["familias_cadastradas"]
lixo = lixo.sort_values("lixo_porcentagem", ascending=False)

,cod_municipio,lixo_total,familias_cadastradas,lixo_porcentagem
5101,432377,584,584.0,1.000000
4716,430558,553,553.0,1.000000
4616,430057,361,361.0,1.000000
4796,430955,1536,1536.0,1.000000
3739,354210,4092,4098.0,0.998536
...,...,...,...,...
746,220375,412,6212.0,0.066323
493,210240,551,8952.0,0.061550
472,210135,365,6491.0,0.056232
480,210190,744,20482.0,0.036325


In [163]:
lixo.to_csv(path_to_output + "cad_unico_lixo.csv", index=False)

#### Parede

In [166]:
parede = pd.read_csv(path_to_cad+"paredes.csv", encoding="latin1")
parede.rename(columns={"Código":"cod_municipio",
                        "Referência":"ano",
                        parede.columns[4]:"parede_1",
                        parede.columns[5]:"parede_2",
                        parede.columns[6]:"parede_3"},
                        inplace=True)
parede = parede[["cod_municipio", "ano", "parede_1", "parede_2", "parede_3"]]
parede["ano"] =  parede["ano"].apply(lambda x: int(x[-4:]))
parede = parede.query("ano==2023")
parede["parede_total"] = parede["parede_1"]+parede["parede_2"]+parede["parede_3"]
parede = parede[["cod_municipio", "parede_total"]]
parede = parede.groupby("cod_municipio").sum().reset_index()

parede = parede.merge(cad, on="cod_municipio", how="left")
parede["parede_porcentagem"] = parede["parede_total"]/parede["familias_cadastradas"]
parede = parede.sort_values("parede_porcentagem", ascending=False)

In [168]:
parede.to_csv(path_to_output + "cad_unico_parede.csv", index=False)

#### Piso

In [169]:
piso = pd.read_csv(path_to_cad+"/piso.csv", encoding="latin1")
piso.columns

Index(['Código', 'Unidade Territorial', 'UF', 'Referência', 'Terra', 'Cimento',
       'Madeira  aproveitada', 'Madeira aparelhada', 'Ceramica', 'Carpete',
       'Outro  material', 'Sem resposta'],
      dtype='object')

In [170]:
piso = pd.read_csv(path_to_cad+"/piso.csv", encoding="latin1")
piso.rename(columns={"Código":"cod_municipio",
                        "Referência":"ano",
                        piso.columns[-4]:"piso_1",
                        piso.columns[-5]:"piso_2",
                        piso.columns[-6]:"piso_3"},
                        inplace=True)
piso = piso[["cod_municipio", "ano", "piso_1", "piso_2", "piso_3"]]
piso["ano"] =  piso["ano"].apply(lambda x: int(x[-4:]))
piso = piso.query("ano==2023")
piso["piso_total"] = piso["piso_1"]+piso["piso_2"]+piso["piso_3"]
piso = piso[["cod_municipio", "piso_total"]]
piso = piso.groupby("cod_municipio").sum().reset_index()

piso = piso.merge(cad, on="cod_municipio", how="left")
piso["piso_porcentagem"] = piso["piso_total"]/piso["familias_cadastradas"]

piso.to_csv(path_to_output + "cad_unico_piso.csv", index=False)


,cod_municipio,piso_total,familias_cadastradas,piso_porcentagem
0,110001,9048,14809.0,0.610980
1,110002,34248,51741.0,0.661912
2,110003,1938,2731.0,0.709630
3,110004,38313,52768.0,0.726065
4,110005,5371,9489.0,0.566024
...,...,...,...,...
5565,522200,3451,7927.0,0.435348
5566,522205,1623,5995.0,0.270726
5567,522220,2177,4420.0,0.492534
5568,522230,1790,5004.0,0.357714


In [172]:
piso["piso_porcentagem"].sort_values()

3330    0.000980
3466    0.001252
2651    0.002079
2941    0.002328
2939    0.002887
          ...   
4999    0.992963
4716    0.994575
4376    0.994643
4702    0.994983
4686    0.996089
Name: piso_porcentagem, Length: 5570, dtype: float64